In [172]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib
import sys

In [173]:
# Find the ERO-SNN folder and add it to the python path
current_dir = os.getcwd()

while os.path.basename(current_dir) != 'ERO-SNN':
    print(os.path.basename(current_dir))
    current_dir = os.path.dirname(current_dir)
    
print(f"Found ERO-SNN folder: {current_dir}")
sys.path.append(current_dir)
os.chdir(current_dir)

Found ERO-SNN folder: /home/bernard/Desktop/IIT_Thesis/ERO-SNN


In [174]:
params = {'xtick.labelsize': 16,
         'ytick.labelsize': 16,
         'font.size': 11,
         'figure.autolayout': True,  # similar to tight_layout
         'figure.figsize': [7.2, 4.5],  # ratio 1.6 is pleasant
         'axes.titlesize': 15,
         'axes.labelsize': 15,
         'lines.linewidth': 2,
         'lines.markersize': 4,
         'legend.fontsize': 12}
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.style.use(params)

In [175]:
# Directory where the CSV files are saved
output_dir = 'quantitative_results_matched_kernels'
score_dir = 'scores'

# List of kernel values
kernel_values = [4, 8, 12, 20]

# Define line styles for different kernel values
line_styles = ['-', '--', '-.', ':']


# Function to calculate moving average
def moving_average(data, window_size):
    return data.rolling(window=window_size).mean()


# Create output directory if it doesn't exist
if not os.path.exists(score_dir):
    os.makedirs(score_dir)


# Get the list of datasets and score types
datasets = ('MVSEC', 'h36m')
dataset_name = {'MVSEC':'MVSEC Outdoor Day','h36m':'Event-Human3.6m'}
score_types = ('ssim' ,'nmi')


In [176]:
# Colors for different kernel values
colors = ['b', 'orange', 'g']

# Initialize handles and labels for the legend
handles = []
labels = []

# Calculate axis limits for each score_type
y_limits = {score_type:{'y_min': float('inf'), 'y_max': float('-inf')} for score_type in score_types}
x_limits = {dataset:{'x_min': float('inf'), 'x_max': float('-inf')} for dataset in datasets}

# First pass to determine axis limits
for dataset in datasets:
    for score_type in score_types:
        for kernel_value in kernel_values:
            for filename in os.listdir(output_dir):
                if filename.startswith(dataset) and f'NN_{kernel_value}' in filename and filename.endswith(f'{score_type}_scores.csv'):
                    filepath = os.path.join(output_dir, filename)
                    df = pd.read_csv(filepath)
                    df_ma = df.apply(lambda x: moving_average(x, 10))
                    y_limits[score_type]['y_min'] = min(y_limits[score_type]['y_min'], df_ma.min().min())
                    y_limits[score_type]['y_max'] = max(y_limits[score_type]['y_max'], df_ma.max().max())
                    x_limits[dataset]['x_min'] = min(x_limits[dataset]['x_min'], df_ma.index.min())
                    x_limits[dataset]['x_max'] = max(x_limits[dataset]['x_max'], df_ma.index.max())

# Second pass to generate plots
for dataset in datasets:
    for score_type in score_types:
        plt.figure(figsize=(10, 6))
        for kernel_value, line_style in zip(kernel_values, line_styles):
            for filename in os.listdir(output_dir):
                if filename.startswith(dataset) and f'NN_{kernel_value}' in filename and filename.endswith(f'{score_type}_scores.csv'):
                    print(f'Processing {filename}')
                    filepath = os.path.join(output_dir, filename)
                    df = pd.read_csv(filepath)
                    df_ma = df.apply(lambda x: moving_average(x, 10))
                    for column, color in zip(df_ma.columns, colors):
                        if 'SCM' in column:
                            # Replace SCM with ESCM
                            column_label = column.replace('SCM', 'ESCM')
                        else:
                            column_label = column
                        handle, = plt.plot(df_ma[column], label=f'{column_label} (k={kernel_value})', linestyle=line_style, color=color)
                        if f'{column_label} (k={kernel_value})' not in labels:
                            handles.append(handle)
                            labels.append(f'{column_label} (k={kernel_value})')

        score_type = score_type.upper()
        plt.title(f'{score_type} scores for {dataset_name[dataset]}')
        plt.xlabel('Frame Index')
        plt.ylabel(score_type)
        # plt.xlim(round(x_limits[dataset]['x_min'], 1), round(x_limits[dataset]['x_max'], 1))
        if score_type == 'SSIM':
            plt.ylim(0, 1)
        else:
            # plt.ylim(round(y_limits[score_type.lower()]['y_min'], 1), round(y_limits[score_type.lower()]['y_max'], 1))
            plt.ylim(1, 1.2)
        plt.grid(True)
        plt.xlim(left=0)

        # Save the combined plot
        plot_filename = f"{dataset}_{score_type}_combined.png"
        plot_filepath = os.path.join(score_dir, plot_filename)
        plt.savefig(plot_filepath)
        plt.close()

# Create a separate figure for the legend
fig_legend = plt.figure(figsize=(20, 1))
fig_legend.legend(handles, labels, ncol=len(kernel_values), loc='center')
legend_filename = f"scores_legend.png"
legend_filepath = os.path.join(score_dir, legend_filename)
fig_legend.savefig(legend_filepath)
plt.close(fig_legend)

Processing MVSEC_NN_4_ssim_scores.csv
Processing MVSEC_NN_8_ssim_scores.csv
Processing MVSEC_NN_12_ssim_scores.csv
Processing MVSEC_NN_20_ssim_scores.csv
Processing MVSEC_NN_4_nmi_scores.csv
Processing MVSEC_NN_8_nmi_scores.csv
Processing MVSEC_NN_12_nmi_scores.csv
Processing MVSEC_NN_20_nmi_scores.csv
Processing h36m_NN_4_ssim_scores.csv
Processing h36m_NN_8_ssim_scores.csv
Processing h36m_NN_12_ssim_scores.csv
Processing h36m_NN_20_ssim_scores.csv
Processing h36m_NN_4_nmi_scores.csv
Processing h36m_NN_8_nmi_scores.csv
Processing h36m_NN_12_nmi_scores.csv
Processing h36m_NN_20_nmi_scores.csv


In [177]:
params = {'xtick.labelsize': 16,
         'ytick.labelsize': 16,
         'font.size': 11,
         'figure.autolayout': True,  # similar to tight_layout
         'figure.figsize': [7.2, 4.5],  # ratio 1.6 is pleasant
         'axes.titlesize': 15,
         'axes.labelsize': 15,
         'lines.linewidth': 2,
         'lines.markersize': 4,
         'legend.fontsize': 15}
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.style.use(params)

In [178]:
# # Generate combined plots for each dataset and score type pair
# for dataset in datasets:
#     fig, axes = plt.subplots(1, 2,  figsize=(20, 6))  # Create a figure with 2 subplots
#     for ax, score_type in zip(axes, score_types):
#         for kernel_value, line_style in zip(kernel_values, line_styles):
#             for filename in os.listdir(output_dir):
#                 if filename.startswith(dataset) and f'NN_{kernel_value}' in filename and filename.endswith(f'{score_type}_scores.csv'):
#                     print(f'Processing {filename}')
#                     filepath = os.path.join(output_dir, filename)
#                     df = pd.read_csv(filepath)
#                     df_ma = df.apply(lambda x: moving_average(x, 10))
#                     for column, color in zip(df_ma.columns, colors):
#                         if 'SCM' in column:
#                             # Replace SCM with ESCM
#                             column_label = column.replace('SCM', 'ESCM')
#                         else:
#                             column_label = column
#                         ax.plot(df_ma[column], label=f'{column_label} (k={kernel_value})', linestyle=line_style, color=color)
        
#         score_type = score_type.upper()
#         ax.set_title(f'{score_type} scores for {dataset}')
#         ax.set_xlabel('Frame')
#         ax.set_ylabel(score_type)
#         handles, labels = ax.get_legend_handles_labels()
#         ax.grid(True)
        
#     plt.subplots_adjust(bottom=0.2, hspace=0.5)
#     fig.legend(handles, labels, ncol=len(kernel_values), bbox_to_anchor=(0.5, -0.2), loc='lower center')

#     # # Save the combined plot
#     # plot_filename = f"{dataset}_combined_test.png"
#     # plot_filepath = os.path.join(score_dir, plot_filename)
#     # plt.savefig(plot_filepath)
#     # plt.close()